<a href="https://colab.research.google.com/github/Zlmknc/ML-CKD/blob/main/ckd_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fetaure Extraction with PCA

In [34]:
from sklearn.decomposition import PCA
import numpy as np


# Başlamadan önce: X_scaled ve y'nin tanımlı olması gerekiyor.
print("\n--- PCA ile Öznitelik Dönüşümü Başlıyor ---")

# PCA nesnesini oluştur (önce tüm bileşenleri hesaplatalım)
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Açıklanan kümülatif varyans oranlarını al
cum_var = np.cumsum(pca.explained_variance_ratio_)
print("Kümülatif Varyans Oranları:", cum_var)

# %95'lik varyansı açıklayan bileşen sayısını bul
target_variance = 0.95
n_components = np.argmax(cum_var >= target_variance) + 1
print(f"Varyansın %{target_variance*100:.1f}'ini açıklamak için {n_components} bileşen seçildi.")

# Bu bileşen sayısıyla PCA'yı yeniden uygula
pca_final = PCA(n_components=n_components)
X_reduced = pca_final.fit_transform(X_scaled)

# Sonuçları yazdır
print(f"Seçilen bileşenlerin açıkladığı toplam varyans: {np.sum(pca_final.explained_variance_ratio_)*100:.2f}%")
print(f"PCA sonrası veri boyutu: {X_reduced.shape}")


--- PCA ile Öznitelik Dönüşümü Başlıyor ---
Kümülatif Varyans Oranları: [0.28182563 0.35613191 0.42572039 0.47913812 0.53042818 0.57766134
 0.62143115 0.66164249 0.70049171 0.73390334 0.76664578 0.79835729
 0.82555189 0.85006009 0.87346452 0.89437604 0.91272445 0.93051868
 0.9448641  0.95901387 0.9713404  0.98266579 0.9921912  1.        ]
Varyansın %95.0'ini açıklamak için 20 bileşen seçildi.
Seçilen bileşenlerin açıkladığı toplam varyans: 95.90%
PCA sonrası veri boyutu: (400, 20)


In [35]:
# --- 2. PCA Sonrası Veriyi Eğitim ve Test Setlerine Ayırma ---
# Model eğitiminde kullanılacak olan X_reduced verisini eğitim ve test setlerine ayırıyoruz.
X_train_pca, X_test_pca, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\nEğitim seti boyutu (PCA sonrası): {X_train_pca.shape}")
print(f"Test seti boyutu (PCA sonrası): {X_test_pca.shape}")

# --- 3. Modeller ve Parametre Izgaraları (Aynı Şekilde Kullanılacak) ---
# NOT: MultinomialNB ve BernoulliNB için Pipeline içindeki MinMaxScaler'a dikkat edin.
# PCA sonrası veri, zaten dönüştürülmüş olduğu için bu scaler'lar gereksiz görünebilir.
# Ancak, bu Naive Bayes türleri pozitif giriş beklediği için MinMaxScaler'ı korumak mantıklı olabilir.
# Diğer modeller için 'scaler' adımı pipeline'dan kaldırıldığı için PCA sonrası veri doğrudan kullanılır.
model_param_list = [
    # ✅ K-Nearest Neighbors
    {
        'name': 'KNN',
        'pipeline': Pipeline([('model', KNeighborsClassifier())]),
        'param_grid': [
            {
                'model__n_neighbors': [1, 3, 5],
                'model__weights': ['uniform', 'distance'],
                'model__metric': ['euclidean', 'manhattan']
            },
            {
                'model__n_neighbors': [1, 3, 5],
                'model__weights': ['uniform', 'distance'],
                'model__metric': ['minkowski'],
                'model__p': [1, 2, 3, 4]
            }
        ]
    },
    # ✅ Naive Bayes
    {
        'name': 'Naive Bayes - GaussianNB',
        'pipeline': Pipeline([('model', GaussianNB())]),
        'param_grid': {}
    },
    {
        'name': 'Naive Bayes - MultinomialNB',
        # PCA sonrası bileşenler negatif değerler içerebilir. MultinomialNB pozitif bekler.
        # Bu nedenle MinMaxScaler'ı pipeline'da tutmak hala önemlidir.
        'pipeline': Pipeline([('scaler', MinMaxScaler()), ('model', MultinomialNB())]),
        'param_grid': {
            'model__alpha': [0.1, 0.5, 1.0, 2.0]
        }
    },
    {
        'name': 'Naive Bayes - BernoulliNB',
        # Aynı şekilde BernoulliNB de ikili veya [0,1] değerler bekler, MinMaxScaler gerekli.
        'pipeline': Pipeline([('scaler', MinMaxScaler()), ('model', BernoulliNB())]),
        'param_grid': {
            'model__alpha': [0.1, 0.5, 1.0, 2.0],
            'model__binarize': [0.0, 0.5, 1.0]
        }
    },
    # ✅ Decision Tree
    {
        'name': 'Decision Tree',
        'pipeline': Pipeline([('model', DecisionTreeClassifier())]), # Ağaç tabanlı modeller ölçekleme gerektirmez
        'param_grid': {
            'model__criterion': ['gini', 'entropy', 'log_loss'],
            'model__splitter': ['best', 'random'],
            'model__max_depth': [None, 5, 10, 20, 50],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__max_features': [None, 'sqrt', 'log2'],
            'model__max_leaf_nodes': [None, 10, 20, 50]
        }
    },
    # ✅ SVM - RBF Kernel
    {
        'name': 'SVM - RBF Kernel',
        'pipeline': Pipeline([('model', SVC(probability=True))]),
        'param_grid': {
            'model__C': [0.1, 1, 10, 100],
            'model__gamma': [1, 0.1, 0.01, 0.001],
            'model__kernel': ['rbf']
        }
    },
    # ✅ SVM - Polynomial Kernel
    {
        'name': 'SVM - Polynomial Kernel',
        'pipeline': Pipeline([('model', SVC(kernel='poly', probability=True))]),
        'param_grid': {
            'model__C': [0.1, 1, 10],
            'model__degree': [2, 3, 4],
            'model__gamma': ['scale', 'auto'],
            'model__coef0': [0, 1]
        }
    },
    # ✅ Logistic Regression
    {
        'name': 'Logistic Regression',
        'pipeline': Pipeline([('model', LogisticRegression(solver='liblinear', max_iter=1000))]),
        'param_grid': {
            'model__C': [0.01, 0.1, 1, 10, 100],
            'model__penalty': ['l1', 'l2'],
            'model__class_weight': [None, 'balanced']
        }
    },
    # ✅ Random Forest
    {
        'name': 'Random Forest',
        'pipeline': Pipeline([('model', RandomForestClassifier())]), # Ağaç tabanlı modeller ölçekleme gerektirmez
        'param_grid': {
            'model__n_estimators': [50, 100, 200],
            'model__max_depth': [None, 5, 10, 20],
            'model__min_samples_split': [2, 5],
            'model__min_samples_leaf': [1, 2],
            'model__bootstrap': [True, False]
        }
    },
    # ✅ MLP
    {
        'name': 'MLP',
        'pipeline': Pipeline([('model', MLPClassifier(max_iter=1000))]),
        'param_grid': {
            'model__hidden_layer_sizes': [(50,), (100,), (50, 50)],
            'model__activation': ['relu', 'tanh'],
            'model__solver': ['adam', 'sgd'],
            'model__alpha': [0.0001, 0.001],
            'model__learning_rate': ['constant', 'adaptive']
        }
    }
]

# --- 4. Model Değerlendirme Fonksiyonu (Daha Önceki Fonksiyonun Geliştirilmiş Hali) ---
def evaluate_models_full_metrics(models_params, X_train_data, y_train_data, X_test_data, y_test_data, cv_folds=10):
    """
    Verilen model listesini çapraz doğrulama ile değerlendirir,
    test seti üzerinde çeşitli metrikleri hesaplar ve ROC eğrisi verilerini döndürür.
    Her model için ayrıntılı performans metriklerini konsola yazdırır.

    Args:
        models_params (list): Her biri model adı, pipeline ve param_grid içeren sözlüklerin listesi.
        X_train_data (array-like): Eğitim özellik seti.
        y_train_data (array-like): Eğitim hedef değişkeni.
        X_test_data (array-like): Test özellik seti.
        y_test_data (array-like): Test hedef değişkeni.
        cv_folds (int): GridSearchCV için çapraz doğrulama katlamalarının sayısı.

    Returns:
        tuple: (pd.DataFrame sonuç tablosu, ROC eğrisi verileri için sözlük)
    """
    results = []
    roc_data = {} # ROC eğrisi verilerini saklamak için

    # Sınıf dengesini katlamalar arasında korumak için StratifiedKFold kullanılır
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)

    for model_info in models_params:
        model_name = model_info['name']
        pipeline = model_info['pipeline']
        param_grid = model_info['param_grid']

        print(f"\n--- {model_name} eğitimi başlıyor ({cv_folds} Katlı Çapraz Doğrulama) ---")
        start_time = time.time()

        best_estimator = None
        best_cv_score = "N/A"
        best_params = "N/A"

        try:
            # MultinomialNB ve BernoulliNB için özel durum (MinMaxScaler gerektirir)
            # Bu modellerin pipeline'ında MinMaxScaler zaten olduğu için, X_train_data'yı doğrudan kullanırız.
            # Diğer modeller için PCA sonrası veri doğrudan kullanılacaktır.
            X_train_for_model = X_train_data
            X_test_for_model = X_test_data


            if isinstance(param_grid, dict) and not param_grid:
                # Parametre ızgarası boşsa (örn. GaussianNB), pipeline'ı doğrudan eğit
                print(f"  {model_name}: Boş param_grid, doğrudan eğitim yapılıyor...")
                pipeline.fit(X_train_for_model, y_train_data)
                best_estimator = pipeline
                best_cv_score = pipeline.score(X_train_for_model, y_train_data) # Eğitim seti skoru
                best_params = "N/A (Varsayılan)"
            else:
                grid_search = GridSearchCV(
                    estimator=pipeline,
                    param_grid=param_grid,
                    cv=cv, # Çapraz doğrulama katlama sayısı
                    scoring='accuracy',
                    n_jobs=-1,
                    verbose=0
                )
                print(f"  {model_name}: GridSearchCV çalıştırılıyor...")
                grid_search.fit(X_train_for_model, y_train_data)
                best_estimator = grid_search.best_estimator_
                best_cv_score = grid_search.best_score_
                best_params = grid_search.best_params_
                print(f"  {model_name}: En iyi skor: {best_cv_score:.4f}, En iyi parametreler: {best_params}")

                # Best 5 parameter combinations if GridSearchCV was run
                if hasattr(grid_search, 'cv_results_'):
                    # Map params to their mean test scores for sorting
                    param_scores = {tuple(sorted(p.items())): grid_search.cv_results_['mean_test_score'][i]
                                    for i, p in enumerate(grid_search.cv_results_['params'])}
                    # Sort and get top 5 unique param combinations
                    sorted_params_with_scores = sorted(param_scores.items(), key=lambda item: -item[1])
                    top_5_params = []
                    seen_params = set()
                    for p_tuple, _ in sorted_params_with_scores:
                        current_params = dict(p_tuple)
                        if tuple(sorted(current_params.items())) not in seen_params:
                            top_5_params.append(current_params)
                            seen_params.add(tuple(sorted(current_params.items())))
                        if len(top_5_params) == 5:
                            break

                    if top_5_params: # Only print if there are valid parameters
                        print("📌 En iyi 5 parametre kombinasyonu:")
                        for i, p in enumerate(top_5_params, 1):
                            print(f"{i}. {p}")


            # Test seti üzerinde tahminler yapma
            y_pred = best_estimator.predict(X_test_for_model)

            # ROC AUC için olasılık skorları
            y_pred_proba = None
            if hasattr(best_estimator, 'predict_proba'):
                y_pred_proba = best_estimator.predict_proba(X_test_for_model)[:, 1]
            else:
                print(f"  Uyarı: {model_name} için predict_proba mevcut değil. ROC AUC hesaplanamayacak.")

            # Metrikleri hesapla
            accuracy = accuracy_score(y_test_data, y_pred)
            precision = precision_score(y_test_data, y_pred, zero_division=0)
            recall = recall_score(y_test_data, y_pred, zero_division=0)
            f1 = f1_score(y_test_data, y_pred, zero_division=0)
            mcc = matthews_corrcoef(y_test_data, y_pred)

            cm = confusion_matrix(y_test_data, y_pred)
            tn, fp, fn, tp = cm.ravel()
            specificity = tn / (tn + fp) if (tn + fp) != 0 else 0 # Handle division by zero

            # ROC AUC
            if y_pred_proba is not None:
                roc_auc = roc_auc_score(y_test_data, y_pred_proba)
                fpr, tpr, thresholds = roc_curve(y_test_data, y_pred_proba)
                roc_data[model_name] = {'fpr': fpr, 'tpr': tpr, 'roc_auc': roc_auc}
            else:
                roc_auc = "N/A"
                roc_data[model_name] = {'fpr': None, 'tpr': None, 'roc_auc': "N/A"}

            status = "Başarılı"

            # Print detailed metrics to console
            print("📈 Performans Metrikleri (Test Seti):")
            print(f"Accuracy: {accuracy:.4f}")
            print(f"Precision: {precision:.4f}")
            print(f"Recall: {recall:.4f}")
            print(f"F1 Score: {f1:.4f}")
            print(f"Specificity: {specificity:.4f}")
            print(f"MCC: {mcc:.4f}")
            print(f"ROC AUC: {roc_auc:.4f}" if isinstance(roc_auc, float) else f"ROC AUC: {roc_auc}")
            print("\nConfusion Matrix:")
            print(f"True Negatives (TN): {tn}")
            print(f"False Positives (FP): {fp}")
            print(f"False Negatives (FN): {fn}")
            print(f"True Positives (TP): {tp}")
            print("\nClassification Report:")
            print(classification_report(y_test_data, y_pred, zero_division=0))


        except Exception as e:
            print(f"  Hata oluştu {model_name} eğitimi sırasında: {e}")
            accuracy, precision, recall, f1, mcc, specificity, roc_auc = "Hata", "Hata", "Hata", "Hata", "Hata", "Hata", "Hata"
            best_cv_score, best_params = "Hata", "Hata"
            status = "Hata"
            roc_data[model_name] = {'fpr': None, 'tpr': None, 'roc_auc': "Hata"}
            # Print error details for metrics if an error occurred
            print("📈 Performans Metrikleri (Test Seti): Hata oluştu.")
            print("Confusion Matrix: Hata oluştu.")
            print("Classification Report: Hata oluştu.")


        end_time = time.time()
        training_time = end_time - start_time

        results.append({
            'Model Adı': model_name,
            'Durum': status,
            'Eğitim Süresi (s)': f"{training_time:.2f}",
            'GridSearchCV En İyi Skor (Doğruluk)': f"{best_cv_score:.4f}" if isinstance(best_cv_score, float) else best_cv_score,
            'Doğruluk (Test)': f"{accuracy:.4f}" if isinstance(accuracy, float) else accuracy,
            'Kesinlik (Test)': f"{precision:.4f}" if isinstance(precision, float) else precision,
            'Duyarlılık (Test)': f"{recall:.4f}" if isinstance(recall, float) else recall,
            'Özgüllük (Test)': f"{specificity:.4f}" if isinstance(specificity, float) else specificity,
            'F1 Skoru (Test)': f"{f1:.4f}" if isinstance(f1, float) else f1,
            'MCC (Test)': f"{mcc:.4f}" if isinstance(mcc, float) else mcc,
            'ROC AUC (Test)': f"{roc_auc:.4f}" if isinstance(roc_auc, float) else roc_auc,
            'En İyi Parametreler': str(best_params)
        })
        print(f"--- {model_name} eğitimi tamamlandı. Süre: {training_time:.2f} saniye ---\n")

    return pd.DataFrame(results), roc_data

# --- 5. Model Değerlendirme Fonksiyonunu Çalıştırma (PCA Sonrası Veri İle) ---
print("\n--- PCA ile Azaltılmış Veri Üzerinde Model Eğitimi Başlıyor ---")
results_df, roc_curve_data = evaluate_models_full_metrics(
    model_param_list,
    X_train_pca, # Burada PCA sonrası eğitim verisi kullanılıyor
    y_train,
    X_test_pca,  # Burada PCA sonrası test verisi kullanılıyor
    y_test,
    cv_folds=10 # Çapraz doğrulama katlaması
)

# --- 6. Nihai Sonuçları Yazdırma ---
print("\n--- Tüm Modeller İçin Özet Değerlendirme Sonuçları (PCA Sonrası Veri) ---")
print(results_df)


Eğitim seti boyutu (PCA sonrası): (280, 20)
Test seti boyutu (PCA sonrası): (120, 20)

--- PCA ile Azaltılmış Veri Üzerinde Model Eğitimi Başlıyor ---

--- KNN eğitimi başlıyor (10 Katlı Çapraz Doğrulama) ---
  KNN: GridSearchCV çalıştırılıyor...
  KNN: En iyi skor: 0.9786, En iyi parametreler: {'model__metric': 'euclidean', 'model__n_neighbors': 1, 'model__weights': 'uniform'}
📌 En iyi 5 parametre kombinasyonu:
1. {'model__metric': 'euclidean', 'model__n_neighbors': 1, 'model__weights': 'uniform'}
2. {'model__metric': 'euclidean', 'model__n_neighbors': 1, 'model__weights': 'distance'}
3. {'model__metric': 'manhattan', 'model__n_neighbors': 1, 'model__weights': 'uniform'}
4. {'model__metric': 'manhattan', 'model__n_neighbors': 1, 'model__weights': 'distance'}
5. {'model__metric': 'minkowski', 'model__n_neighbors': 1, 'model__p': 1, 'model__weights': 'uniform'}
📈 Performans Metrikleri (Test Seti):
Accuracy: 0.9833
Precision: 1.0000
Recall: 0.9733
F1 Score: 0.9865
Specificity: 1.0000
MC